In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [2]:
example = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\SLAT\memory_disorder_SLAT\regex_dataset.xlsx", sheet_name = "Sheet1", index_col = 0)

In [3]:
example.head()

,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
PatientID,,,,,,,,,,
Z6708893,1.004451e+10,100100942.0,3.115772e+09,Progress Notes,1000021019,955432079,2016-04-28,----------------------------------------------...,"""Visit Other Routine general medical examinati...","[""memory change""]"
Z7766129,1.002544e+10,100240713.0,3.116814e+09,H&P,1000431290,954962537,2016-04-28,----------------------------------------------...,"""\u2022 Doxepin Other (See Comments) vivid dre...","[""memory problem""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""accident moving next year due to extra bedroo...","[""dementia""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""techniques discuss again correlations long te...","[""dementia""]"
Z9132953,2.511694e+06,105740146.0,3.107134e+09,Progress Notes,1001733204,956198896,2016-04-29,----------------------------------------------...,"""benzo rxs 4/98, 9/97, 5/04 Family history: -m...","[""dementia""]"
